In [1]:
import requests
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
import matplotlib.pyplot as plt
import numpy as np
# import contextily as cx

In [9]:
# 1. Fetch data from your API
# Replace with your specific region endpoint if needed
URL = "http://127.0.0.1:8000/geojson/all/final_score/dolnoslaskie" 
response = requests.get(URL)

In [10]:
if response.status_code == 200:
    data = response.json()
    
    # 2. Convert to a standard Pandas DataFrame
    df = pd.DataFrame(data)
    
    # 3. Clean and Parse Geometries
    # Remove any rows where geometry is missing to avoid errors
    df = df.dropna(subset=['geometry'])
    df['geometry'] = df['geometry'].apply(lambda x: shape(x))
    
    # 4. Convert to a GeoDataFrame
    # Set original CRS (EPSG:2180 for Poland)
    gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:2180")
    
    # # 5. Export back to a .geojson file
    # gdf.to_file("exported_data.geojson", driver='GeoJSON')
    # print("Successfully converted to GeoDataFrame and exported to file!")

In [11]:
gdf.head(3)

,dni_score,pvout_score,id,box_id,dso_score,station_score,land_score,area,region_name,temp_score,dem_score,solar_score,road_score,fclass,perimeter,geometry
0,960.610688,1076.803158,21086,1491,0.764648,0.464377,0.082329,89169.514306,dolnoslaskie,8.881754,263.925170,0.240452,0.976676,scrub,2406.814298,"MULTIPOLYGON (((208981.059 341846.36, 208973.4..."
1,999.975966,NaN,21124,3832,0.825775,0.939360,NaN,2236.226982,dolnoslaskie,NaN,179.750000,0.172022,0.927639,None,253.859387,"MULTIPOLYGON (((220945.353 376860.797, 220942...."
2,1016.105286,1108.899048,718,40066,0.564122,0.586415,0.268502,1000000.000000,dolnoslaskie,9.350000,193.046441,0.132841,0.980395,farmland,4000.000000,"MULTIPOLYGON (((402981.181 376880.219, 403981...."


In [12]:
gdf_reduced = gdf.set_index('id', inplace=True)
criteria= [
    'dni_score', 'temp_score', 'pvout_score', 'dem_score', 
    'road_score', 'station_score', 'solar_score', 'dso_score', 'land_score'
]

gdf_reduced = gdf[criteria]
types = np.array([+1, +1, +1, -1, +1, +1, -1, +1, +1]) 

In [13]:
df_reduced = gdf_reduced.dropna(subset=criteria)
df_reduced.head(3)

,dni_score,temp_score,pvout_score,dem_score,road_score,station_score,solar_score,dso_score,land_score
id,,,,,,,,,
21086,960.610688,8.881754,1076.803158,263.925170,0.976676,0.464377,0.240452,0.764648,0.082329
718,1016.105286,9.350000,1108.899048,193.046441,0.980395,0.586415,0.132841,0.564122,0.268502
737,1020.041677,9.400000,1113.281982,187.013666,0.968086,0.636506,0.106032,0.469523,0.709701


In [14]:
# columns already scaled to [0,1] and benefit-type; leave them untouched
already_scaled = {'land_score', 'dso_score', 'road_score', 'station_score', 'solar_score'}

#build normalized matrix Xn (selective min–max)
X = df_reduced[criteria].to_numpy(float)
Xn = np.empty_like(X, float)

for j, colname in enumerate(criteria):
    col = X[:, j]

    if colname in already_scaled:
        # already in [0,1]; invert only if it is a cost criterion
        Xn[:, j] = 1 - col if types[j] == -1 else col

    else:
        cmin, cmax = np.nanmin(col), np.nanmax(col)

        if np.isclose(cmax, cmin):
            Xn[:, j] = 0.0
        else:
            # min–max with cost handling
            Xn[:, j] = (col - cmin) / (cmax - cmin) if types[j] == +1 else (cmax - col) / (cmax - cmin)

# CRITIC weights
sigma = np.nanstd(Xn, axis=0, ddof=1)
R = np.corrcoef(Xn + 1e-12*np.random.randn(*Xn.shape), rowvar=False)
R = np.clip(R, -1.0, 1.0)
C = sigma * np.sum(1.0 - np.abs(R), axis=1)

w = np.ones_like(C)/len(C) if np.allclose(C.sum(), 0.0) else C / C.sum()

weights_df = pd.DataFrame({'Criterion': criteria, 'CRITIC_Weight': w}).sort_values('CRITIC_Weight', ascending=False)
display(weights_df.style.background_gradient(cmap='Blues').format({'CRITIC_Weight':'{:.4f}'}).set_caption("CRITIC Weights"))

,Criterion,CRITIC_Weight
8,land_score,0.2768
7,dso_score,0.1368
5,station_score,0.1307
6,solar_score,0.1287
1,temp_score,0.0923
3,dem_score,0.0780
0,dni_score,0.0596
2,pvout_score,0.0556
4,road_score,0.0414


In [ ]:
# --- OPTION A: Static Plot (Quick View) ---
print("Generating Static Plot...")
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
gdf.plot(column='dni_score', ax=ax, legend=True, cmap='viridis', 
            missing_kwds={'color': 'lightgrey'})
cx.add_basemap(ax, crs=gdf.crs, source=cx.providers.OpenStreetMap.Mapnik)
plt.title("Solar Score Distribution")
plt.show()

In [ ]:
print("Generating Grid with OSM Background...")

# 1. Create the figure and axis
fig, ax = plt.subplots(1, 1, figsize=(12, 12))

# 2. Plot the grid boxes
# Use facecolor='none' for transparent boxes, or alpha=0.3 for semi-transparent
gdf.plot(ax=ax, 
         facecolor='none', 
         edgecolor='red', 
         linewidth=0.8, 
         label="Grid Boxes")

# 3. Add the OSM Basemap
# We pass the gdf.crs so contextily knows how to reproject the tiles
cx.add_basemap(ax, crs=gdf.crs, source=cx.providers.OpenStreetMap.Mapnik)

# 4. Final touches
plt.title("Solar Grid Layout over OpenStreetMap")
ax.set_axis_off() # Optional: Hide the XY coordinate labels
plt.show()

In [ ]:
# 1. CRITICAL: Interactive maps require the WGS84 projection (GPS coordinates)
# We convert from your current CRS (2180) to 4326
gdf_interactive = gdf.to_crs(epsg=4326)

# 2. Use the .explore() method
m = gdf_interactive.explore(
    column='dni_score',     # Color the boxes by this column
    cmap='viridis',         # Color palette
    legend=True,            # Show the color scale
    tooltip=['box_id','dni_score'],       # Show box_id when you hover
    popup=True,             # Show all data in a table when you click
    tiles="OpenStreetMap",  # The background map
    style_kwds=dict(color="black", weight=0.5) # The grid line style
)

# Save the map to an HTML file
m.save("map_debug.html")

# # Then, if you are on a local machine, you can open it automatically
# import webbrowser
# webbrowser.open("map_debug.html")